In [2]:
import pickle
from MARBLE import utils

In [ ]:
pos, vel = pickle.load(open('../outputs/spiking_data/data_object.pkl','rb'))

In [ ]:
data = utils.construct_dataset(pos, features=vel, graph_type='cknn', k=40, stop_crit=0.05, 
                                   n_geodesic_nb=10, compute_cl=True, vector=False)

In [ ]:
axes = plotting.fields(data, col=4, alpha=1, width=7, scale=2, titles=titles, view=[70,20])

In [23]:
rates =  pickle.load(open('../outputs/spiking_data/rate_data.pkl','rb'))       

In [24]:
def start_at_gocue(rates, t = 500):
    
    days, conditions = rates.keys(), rates[0].keys()
    
    for d in days:
        for c in conditions:
            rates[d][c] = rates[d][c][:,:,t:]
        
    return rates

In [25]:
rates = start_at_gocue(rates, t = 500)

In [27]:
def fit_pca(rates, pca_n = 3):
    
    days, conditions = rates.keys(), rates[0].keys()
    
    pos = []
    for d in days:
        for c in conditions:
            trials = rates[d][c]
            for t in trials:
                pos.append(t.T)
                
    # fit pca
    pca = PCA(n_components=pca_n)
    pca.fit(np.vstack(pos))
    print(pca.explained_variance_ratio_)
    
    return pca

In [28]:
pca = fit_pca(rates)

[0.31895829 0.08622739 0.07281958]


In [51]:
def compute_velocity(rates, pca):
    
    days, conditions = rates.keys(), rates[0].keys()
    
    # create empty list of lists for each condition
    pos = [[] for u in range(len(conditions))]
    vel = [[] for u in range(len(conditions))]      
      
    # loop over each day
    for d in days:
     
        # loop over conditions
        for i, c in enumerate(conditions):
            
            trials = rates[d][c]   
                       
            # loop over all trials
            for t in trials:
                p = pca.transform(t.T)
                v = np.diff(p, axis=0)
                p = p[:-1,:]
                
                pos[i].append(p)
                vel[i].append(v)
    
    return pos, vel

In [52]:
pos, vel = compute_velocity(rates, pca)

In [53]:
def remove_outliers(pos, vel):
        
    for i, (p_c, v_c) in tqdm(enumerate(zip(pos, vel))): 
        for j, (p, v) in enumerate(zip(p_c, v_c)):
    
            p, v = _remove_outliers(p, v)
            pos[i][j], vel[i][j] = p, v
    
    return pos, vel


def _remove_outliers(pos, vel):
    """  function for removing outliers """
    clf = LocalOutlierFactor(n_neighbors=10)        
    # remove positional outliers
    outliers = clf.fit_predict(pos)
    vel = vel[outliers==1]
    pos = pos[outliers==1]            
    # remove velocity outliers
    outliers = clf.fit_predict(vel)
    vel = vel[outliers==1]
    pos = pos[outliers==1]         
    return pos, vel

In [54]:
remove_outliers(pos, vel)

5it [00:49,  9.96s/it]


KeyboardInterrupt: 

In [8]:
pca = main()

100%|███████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 16749.81it/s]


[0.29664557 0.10035803 0.0684394 ]


In [9]:
def main(pca):        
    # create empty list of lists for each condition
    pos = [[] for u in range(len(conditions))]
    vel = [[] for u in range(len(conditions))]      
      
    # loop over each day
    for day in tqdm(days):
     
        # loop over conditions
        for c, cond in enumerate(conditions):
            
            # go cue at 500ms 
            data = rates[day][cond][:,:,500:]           
                       
            # loop over all trials
            for t in range(data.shape[0]):
                
                # extra trial
                trial = data[t,:,:]
                
                # transform to time x channels
                trial = trial.T
                
                # embed in fitted pca space
                trial = pca.transform(trial)
                
                # take all points except last
                pos[c].append(trial[:-1,:])
                
                # extract vectors between coordinates
                vel[c].append(get_vector_array(trial))
                
        # remove outliers
        if rm_outliers:
            pos[c], vel[c] = remove_outliers(pos[c], vel[c])    
                        
    # stack the trials within each condition
    pos = [np.vstack(u) for u in pos] # trials x time x channels
    vel = [np.vstack(u) for u in vel] # trials x time x channels
    
#with open('../outputs/spiking_data/dataobject.pkl', 'wb') as handle:
#    pickle.dump([pos, vel], handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [ ]:
main()